# <div style="font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:200%; 
text-align:center;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0">Santa 2023 - 3x3x3 Optimal</div>
#### <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:150%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >TABLE OF CONTENTS<br><div>
* [IMPORTS](#1)
* [LOAD DATA](#2)
* [FUNCTIONS](#3)
* [SOLVE](#4)

Using repo from dwalton76: https://github.com/dwalton76/rubiks-cube-NxNxN-solver/tree/master

The solver can attempt optimal solutions for 3x3x3

<a id="1"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" > IMPORTS<br><div> 

In [1]:
import zipfile
import sqlite3
import random
import pandas as pd
import numpy as np

import os

os.chdir('rubiks-cube-NxNxN-solver')
# Print the current working directory
print("Current Working Directory: ", os.getcwd())

database_file = '../../solutions.db'
solution_method = "dwalton76 3x3x3 optimal"

Current Working Directory:  /Users/seanbearden/PycharmProjects/Kaggle/src/santa-2023/magic-cube/rubiks-cube-NxNxN-solver


<a id="2"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >LOAD DATA<br><div> 

In [2]:
with zipfile.ZipFile('../../../../res/data/santa-2023.zip', 'r') as z:
    
    with z.open('puzzle_info.csv') as f:
        puzzle_info = pd.read_csv(f, index_col = 'puzzle_type')        
                
    with z.open('puzzles.csv') as f:
        puzzles = pd.read_csv(f, index_col='id')
    
    with z.open('sample_submission.csv') as f:
        submission = pd.read_csv(f)

<a id="3"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >FUNCTIONS<br><div>
 

In [3]:
U = ['U', 'F', 'R', 'B', 'L', 'D']
U_dict = {
    'A': 'U',
    'B': 'F',
    'C': 'R',
    'D': 'B',
    'E': 'L',
    'F': 'D',
}
def state2ubl(state):
    state_split = state.split(';')
    dim = int(np.sqrt(len(state_split) // 6))
    dim_2 = dim**2
    
    s = ''.join([U_dict[f] for f in state_split])
    
    return s[:dim_2] + s[2*dim_2:3*dim_2] + s[dim_2:2*dim_2] + s[5*dim_2:] + s[4*dim_2:5*dim_2] + s[3*dim_2:4*dim_2]


In [4]:
def apply_moves(initial_state, moves, mmoves):
    state = initial_state.split(";")
    for move_name in mmoves.split('.'):
        state = [state[i] for i in moves[move_name]]
    
    return ';'.join(state)



In [5]:
def sym_rotations(dim, mmoves, new_state, sol_state, num_wildcards, wildcard=False):
    I = ['.'.join([f'{j}{i}' for i in range(dim)]) for j in ['r', 'd', 'f']]
    manipulations = (
            [''] + 
            I + 
            [i1 + '.' + i2 for i1 in I for i2 in I] +
            [i1 + '.' + i2+ '.' + i3 for i1 in I for i2 in I for i3 in I] +
            [i1 + '.' + i2+ '.' + i3 + '.' + i4 for i1 in I for i2 in I for i3 in I for i4 in I]
    )
    
    for rotation in manipulations:
        temp_state = new_state
        if len(rotation) > 0:
            for move in rotation.split('.'):
                temp_state = ';'.join(list(np.asarray(temp_state.split(';'))[np.array(moves[move])]))

        if wildcard: 
            total_correct = np.sum([sol_state[i] == ns for i, ns in enumerate(temp_state.split(';'))])
            if total_correct >= 6*(dim**2) - num_wildcards:
                print(f'solved id: {id}')
                if len(rotation) > 0:
                    mmoves += '.' + rotation
                break
        else:
            if temp_state == sol_state:
                print(f'solved id: {id}')
                if len(rotation) > 0:
                    mmoves += '.' + rotation
                break
                
    return temp_state, mmoves

In [6]:
def move_translation():
    M = {}
    M["U"] = "-d2"
    M["R"] = "r0"
    M["B"] = "-f2"
    M["F"] = "f0"
    M["L"] = "-r2"
    M["D"] = "d0"


    for m in list(M):
        M[m+"2"] = M[m] + '.' + M[m]
        if "-" in M[m]:
            M[m+"'"] = M[m].replace("-","")
        else:
            M[m+"'"] = '.'.join(["-"+i for i in M[m].split('.')])
    
    return M

def inverse_move_translation():
    M = {}
    
    M["r0"] = "R"
    M["f0"] = "F"
    M["d0"] = "D"
    M["-r0"] = "R'"
    M["-f0"] = "F'"
    M["-d0"] = "D'"
    
    
    M["d2"] = "U'"
    M["f2"] = "B'"
    M["r2"] = "L'"
    M["-d2"] = "U"
    M["-f2"] = "B"
    M["-r2"] = "L"

    
    M["r1"] = "R' L"
    M["f1"] = "F' B"
    M["d1"] = "D' U"
    M["-r1"] = "R L'"
    M["-f1"] = "F B'"
    M["-d1"] = "D U'"
    
    
    return M

In [7]:
def get_moves(allowed_moves):
    moves = eval(allowed_moves)
    for move in list(moves):
        moves['-'+move] = np.argsort(moves[move]).tolist()
    return moves

def flip_diag_faces(M):
    # flip_diag_faces = "x' y Rw U Rw' U' F F U Rw U' Rw' F" # F"
    flip_diag_faces = "Rw U2 Rw' U2 Rw' F Rw2 U2 Rw' U2 Rw U Rw' F'"
    mm = '.'.join([M[m] for m in flip_diag_faces.split(' ')])
    return mm


def swap_faces(M):
    moves = "U F2 D Rw U2 Rw' U2 Rw' F Rw2 U2 Rw' U2 Rw U Rw' F' D' F2 U'"
    mm = '.'.join([M[m] for m in moves.split(' ')])
    return mm

        

In [8]:
def solve(initial_state, solution_state, num_wildcards, moves, verbose=False, best=None):
    init_state = initial_state
    sol_state = solution_state
        
    state = state2ubl(init_state)
    print(f'Starting {id}')
    if best is None:
        output = !./rubiks-cube-solver.py --state $state
    else:
        output = !./rubiks-cube-solver.py --state $state --solution333=$best
    # return output
    sol = None
    print(output[-1])
    if output[-1][:9] == 'Solution:':
        sol = output[-1].split(': ')[1]
    else:
        for n in range(1, 21):
            if 'Solution:' in output[-n]:
                sol = output[-n].split('Solution: ')[1].split('2023-')[0]
                break
    if sol is None:
        print(output[-1])
        return output
                
    mmoves = '.'.join([M[m] for m in sol.split(' ')])
    
    new_state = apply_moves(initial_state, moves, mmoves)
    print(solution_state)
    print(new_state)
    temp_state, mmoves = sym_rotations(dim, mmoves, new_state, solution_state, num_wildcards, wildcard=False)
    # print(temp_state)
    return mmoves

In [9]:
def relabel_3x3x3(id, state):
    # Split the string into individual elements.
    state_list = state.split(';')
    if id in range(140, 150):
        # These puzzles has a straightforward translation
        # For each element, determine the replacement based on the criteria.
        for i, val in enumerate(state_list):
            # Extract the number part from the string like 'N33' -> 33.
            num = int(val[1:])
            # Replace based on the given criteria.
            if 0 <= num <= 8:
                state_list[i] = 'A'
            elif 9 <= num <= 17:
                state_list[i] = 'B'
            elif 18 <= num <= 26:
                state_list[i] = 'C'
            elif 27 <= num <= 35:
                state_list[i] = 'D'
            elif 36 <= num <= 44:
                state_list[i] = 'E'
            elif 45 <= num <= 53:
                state_list[i] = 'F'

    elif id in range(130, 140):
        # edges are unique, and therefore can be identified by examining the 
        # solution state and used for translating the inital state.
        edges = [
            (7, 10), (14, 21), (23, 30), (32, 39),
            (12, 41), (16, 46), (34, 52), (5, 19), 
            (1, 28), (3, 37), (25, 50), (43, 48)
        ]

        translations = {
            "AC": "FB",
            "AD": "FC",
            "AE": "FD",
            "AF": "FE",
    
            "BC": "AB",
            "BD": "AC",
            "BF": "AE",
            "BE": "AD",
    
            "CD": "BC",
            "CF": "BE",
            "DE": "CD",
            "EF": "DE",
            
        }
        
        for e1, e2 in edges:
            c1 = state_list[e1]
            c2 = state_list[e2]
            # Sort the strings
            sorted_strings = sorted([c1, c2])
            # Concatenate them
            concatenated_string = ''.join(sorted_strings)
            t1, t2 = translations[concatenated_string]
            if c1 == sorted_strings[0]:
                state_list[e1] = t1
                state_list[e2] = t2
            else:
                state_list[e1] = t2
                state_list[e2] = t1
    return ';'.join(state_list)  

<a id="4"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >SOLVE<br><div> 

In [13]:
# Connect to the SQLite database
conn = sqlite3.connect(database_file)
cursor = conn.cursor()

for id in range(130, 150):
    row = puzzles.loc[id]
    if row['puzzle_type'][:4] == 'cube':
        dim = int(row['puzzle_type'].split('/')[-1])
        moves = get_moves(puzzle_info.loc[row['puzzle_type'], 'allowed_moves'])
        M = move_translation()
    
        Mi = inverse_move_translation()    
    
        select_query = "SELECT moves FROM solutions WHERE id = ?"
    
        # Execute the query
        cursor.execute(select_query, (id,))
        response = cursor.fetchone()
        best_moves= response[0]
        
        temp = " ".join([Mi[i] for i in best_moves.split('.')]).split(" ")

        best_moves_translated = ' '.join(temp)
        best_moves_translated = f'\"{best_moves_translated}\"'

        
        if 130 <= id <= 149:
            cur_state = relabel_3x3x3(id, row['initial_state'])
            sol_state = relabel_3x3x3(id, row['solution_state'])
        else:
            cur_state = row['initial_state']
            sol_state = row['solution_state']
            
        init_moves = solve(cur_state, sol_state, row['num_wildcards'], moves, verbose=False, 
                           best=best_moves_translated) 
    
        # validation
        state = row['initial_state'].split(";")
        for move_name in init_moves.split('.'):
            state = [state[i] for i in moves[move_name]]
        

        try:
            assert row['solution_state'].split(";") == state
            print('SOLVED"')
            mmoves_length = len(init_moves.split('.'))
            
            select_query = "SELECT count FROM solutions WHERE id = ?"

            # Execute the query
            cursor.execute(select_query, (id,))
            response = cursor.fetchone()
            best_moves_length = response[0]
            print(mmoves_length, best_moves_length)
            if mmoves_length < best_moves_length:
                # Insert the moves into the database
                insert_query = ("INSERT OR REPLACE INTO solutions (id, moves, count, solution_method) VALUES (?, ?, ?, ?)")
                cursor.execute(insert_query, (id, init_moves, mmoves_length, solution_method))
                conn.commit()
        except AssertionError:
            print(f"assertion error for {id}")
            print(state)
            break        
            

# Commit the changes and close the connection
conn.commit()
conn.close()


Starting 130
Solution: R2 U2 F' B' D' R U2 R' B2 U' B U2 D2 B2 L U2 R B2 L B2 U2
A;A;A;A;A;A;A;A;A;B;B;B;B;B;B;B;B;B;C;C;C;C;C;C;C;C;C;D;D;D;D;D;D;D;D;D;E;E;E;E;E;E;E;E;E;F;F;F;F;F;F;F;F;F
B;B;B;B;B;B;B;B;B;C;C;C;C;C;C;C;C;C;A;A;A;A;A;A;A;A;A;E;E;E;E;E;E;E;E;E;F;F;F;F;F;F;F;F;F;D;D;D;D;D;D;D;D;D
solved id: 130
SOLVED"
44 34
Starting 131
Solution: L' D B R F L2 R2 B2 U2 B' D L2 D2 F R2 B D2 F' B2 U2 B
A;A;A;A;A;A;A;A;A;B;B;B;B;B;B;B;B;B;C;C;C;C;C;C;C;C;C;D;D;D;D;D;D;D;D;D;E;E;E;E;E;E;E;E;E;F;F;F;F;F;F;F;F;F
E;E;E;E;E;E;E;E;E;A;A;A;A;A;A;A;A;A;D;D;D;D;D;D;D;D;D;F;F;F;F;F;F;F;F;F;B;B;B;B;B;B;B;B;B;C;C;C;C;C;C;C;C;C
solved id: 131
SOLVED"
37 31
Starting 132
Solution: R U2 F U2 L D2 L D' F' B2 D F2 D2 L2 D2 R2 L' D2 B2 L2 U2
A;A;A;A;A;A;A;A;A;B;B;B;B;B;B;B;B;B;C;C;C;C;C;C;C;C;C;D;D;D;D;D;D;D;D;D;E;E;E;E;E;E;E;E;E;F;F;F;F;F;F;F;F;F
E;E;E;E;E;E;E;E;E;B;B;B;B;B;B;B;B;B;A;A;A;A;A;A;A;A;A;D;D;D;D;D;D;D;D;D;F;F;F;F;F;F;F;F;F;C;C;C;C;C;C;C;C;C
solved id: 132
SOLVED"
43 39
Starting 133
Solution: D'

In [11]:
print(temp)

['B', 'B', "U'", "F'", "R'", "U'", 'L', "B'", 'L', 'U', 'U', 'F', 'L', 'F', 'F', 'U', 'U', 'R', 'R', 'D', 'D', 'R', 'R', "U'", 'F', 'F', "D'", 'B', 'B', 'L', 'L', 'R', "R'", 'L', "L'", 'R', "R'", 'L', "L'", 'F', "F'", 'B', "B'", 'R', "R'", 'L', "L'"]
